1.Import required library

In [21]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras_preprocessing import sequence
from keras.utils import to_categorical
from keras.models import load_model

2.Read dataset and do preprocessing

In [22]:
df = pd.read_csv('/content/drive/MyDrive/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [23]:
df.drop(['Unnamed: 2','Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True) #dropping unwanted columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [24]:
#Count of Spam and Ham values
df.groupby(['v1']).size()

v1
ham     4825
spam     747
dtype: int64

In [25]:
#Label Encoding target column
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [26]:
# Test and train spilit 
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [27]:
# Tokenisation function
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

3.Create Model and 4.Add layers(LSTM,Dense-(Hidden Layers),output)

In [28]:
#creating LSTM model
inputs = Input(name='InputLayer',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FullyConnectedLayer1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='OutputLayer')(layer)
layer = Activation('sigmoid')(layer)

5.Compile the model

In [29]:
model = Model(inputs=inputs,outputs=layer)
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 InputLayer (InputLayer)     [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FullyConnectedLayer1 (Dense  (None, 256)              16640     
 )                                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                             

7.Fit the Model

In [31]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 8s 254ms/step - loss: 0.0081 - accuracy: 0.9976 - val_loss: 0.0566 - val_accuracy: 0.9831
Epoch 2/10
30/30 [==============================] - 9s 293ms/step - loss: 0.0055 - accuracy: 0.9984 - val_loss: 0.0601 - val_accuracy: 0.9884
Epoch 3/10
30/30 [==============================] - 8s 256ms/step - loss: 0.0055 - accuracy: 0.9984 - val_loss: 0.0706 - val_accuracy: 0.9842
Epoch 4/10
30/30 [==============================] - 9s 309ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 0.0791 - val_accuracy: 0.9895
Epoch 5/10
30/30 [==============================] - 10s 326ms/step - loss: 0.0033 - accuracy: 0.9992 - val_loss: 0.0800 - val_accuracy: 0.9863
Epoch 6/10
30/30 [==============================] - 8s 254ms/step - loss: 0.0064 - accuracy: 0.9982 - val_loss: 0.1020 - val_accuracy: 0.9821
Epoch 7/10
30/30 [==============================] - 8s 252ms/step - loss: 0.0029 - accuracy: 0.9992 - val_loss: 0.0899 - val_accuracy: 0.9852
Epoch

8.Save the model

In [32]:
model.save("model_1")

9.Test the Model

In [33]:
test_sequences=tok.texts_to_sequences(X_test)
test_sequences_matrix =sequence.pad_sequences(test_sequences,maxlen=max_len)

In [34]:
accuracy = model.evaluate(test_sequences_matrix,Y_test)
print('Accuracy: {:0.3f}'.format(accuracy[1]))

27/27 [==============================] - 1s 21ms/step - loss: 0.2344 - accuracy: 0.9737
Accuracy: 0.974


In [35]:
y_pred =model.predict(test_sequences_matrix)
print(y_pred[25:40].round(3))

27/27 [==============================] - 1s 21ms/step
[[0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]]


In [36]:
print(Y_test[25:40])

[[0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]]
